In [29]:
!pip install python-crfsuite
!pip install sklearn-crfsuite

import sklearn_crfsuite
import numpy as np
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite
import random

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

###Mount the google drive 
###Read the files and populate the data
###Develop feature extraction algorithm
###USE CRF for prediction

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
#facebook_file = "/content/drive/My Drive/Colab Notebooks/swiggy/data/Telugu_English_Twitter.txt"
#twitter_file = "/content/drive/My Drive/Colab Notebooks/swiggy/data/Telugu_English_FB.txt"

facebook_file = "/content/drive/My Drive/Colab Notebooks/swiggy/data/Hindi_English_Twitter.txt"
twitter_file = "/content/drive/My Drive/Colab Notebooks/swiggy/data/Hindi_English_FB.txt"

#facebook_file = "/content/drive/My Drive/Colab Notebooks/swiggy/data/Bengali_English_Twitter.txt"
#twitter_file = "/content/drive/My Drive/Colab Notebooks/swiggy/data/Bengali_English_FB.txt"



files = [ facebook_file, twitter_file ]

split_ratio = 0.7

In [32]:
def readFiles(files):
  
  list_of_words = []
  list_of_tags = []

  for file in files:
    f = open(file, "r")
    words = []
    tags = []

    if f.mode == 'r':
      fl = f.readlines()
      for x in fl:
        str = x.strip()
        if str != "":
          temp = str.split("\t")
          words.append(temp[0])
          tags.append(temp[1])
        else:
          list_of_words.append(words)
          list_of_tags.append(tags)
          words = []
          tags = []
        
  return list_of_words, list_of_tags


In [33]:
def word2features_ngram(sent, k):  ### actual
    word = sent[k]
    features = [
        'token=%s' % (word)
    ]
    # extracting n-grams, for n=1 to 5
    for i in range(1,6):
        # if the value of n is greater than the word length, we exit the loop
        if i > len(word):
            break
        character_features = [word[j:j+i] for j in range(len(word)-i+1)]
        features.extend([
            # is count of individual n-grams important? is the order important?
            "char-%d-gram=%s" % (i, ' '.join(list(set(character_features))))
        ])
    if k == 0:
        # first word in the sentence
        features.append('BOS')
    else:
        features.extend([
            "-1:word=%s" % (sent[k-1])
        ])
    if i == len(sent):
        # last word in the sentence         
        features.append('EOS')
 
    return features  


In [34]:
def word2features_tags(sentence, index):
  return {
  'word': sentence[index],
  'is_first': index == 0,
  'is_last': index == len(sentence) - 1,
  'is_capitalized': sentence[index].upper() == sentence[index],
  'is_all_caps': sentence[index].upper() == sentence[index],
  'is_all_lower': sentence[index].lower() == sentence[index],
  'prefix-1': sentence[index][0],
  'prefix-2': sentence[index][:2],
  'prefix-3': sentence[index][:3],
  'suffix-1': sentence[index][-1],
  'suffix-2': sentence[index][-2:],
  'suffix-3': sentence[index][-3:],
  'n-grams': [sentence[index][j:j+i] for i in range(1,5) for j in range(len(sentence[index])-i+1)],
  'prev_word': '' if index == 0 else sentence[index - 1],
  'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
  'has_hyphen': '-' in sentence[index],
  'is_numeric': sentence[index].isdigit(),
  'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
  }  


In [35]:
def sent2features_tags(sent):
    # generating features for all the words/tokens in a sentence `sent`    
    return [word2features_tags(sent, i) for i in range(len(sent))]
  

def sent2features_ngram(sent):
    # generating features for all the words/tokens in a sentence `sent`    
    return [word2features_ngram(sent, i) for i in range(len(sent))]
  
  
def sent2langs(sent):
    return [language_label for language_label in sent]

In [36]:
data_words , data_tags = readFiles(files)

#print(data_words)
#print(data_tags)

print(len(data_words))
print(len(data_tags))

random.seed(7)


train_sents = data_words[:int(split_ratio*len(data_words))]
valid_sents = data_words[int(split_ratio*len(data_words)):]

train_tags = data_tags[:int(split_ratio*len(data_tags))]
valid_tags = data_tags[int(split_ratio*len(data_tags)):]


print("# Train sentences: %d" % (len(train_sents)))
print("# Validation sentences: %d" % (len(valid_sents)))

print("# Train Labels: %d" % (len(train_tags)))
print("# Validation Lables: %d" % (len(valid_tags)))


1867
1867
# Train sentences: 1306
# Validation sentences: 561
# Train Labels: 1306
# Validation Lables: 561


In [37]:
%%time
train_features = [sent2features_ngram(sent) for sent in train_sents]
test_features = [sent2features_ngram(sent) for sent in valid_sents]

train_labels = [sent2langs(sent) for sent in train_tags]
test_labels = [sent2langs(sent) for sent in valid_tags]

CPU times: user 448 ms, sys: 6.86 ms, total: 455 ms
Wall time: 458 ms


In [38]:
print(train_sents[0])
print(valid_sents[0])

print((train_features[0]))
print((test_features[0]))

print((train_labels[0]))
print((test_labels[0]))

print(len(train_features))
print(len(test_features))

['Asslaam', 'o', "Alaikum!''-*Kehte", ':', '-*Hain', ':', '-*Agr', ':', '-*Subha', ':', '-*Uth', ':', '-*Kr', ':', '-*Achay', ':', '-*Insaan', ':', '-*Ko', ':', '-*Yaad', ':', '-*Kro', ':', '-*To', ':', '-*Din', ':', '-*Bhi', ':', '-*Acha', '...']
['#316', ':', 'This', 'dates', 'back', 'to', 'few', 'years', 'ago', 'when', 'I', 'went', 'to', 'Europe', 'for', 'my', 'internship', '.', 'To', 'save', 'money', 'for', 'accommodation', ',', 'decided', 'to', 'travel', 'during', 'the', 'night', 'by', 'Eurolines', 'Bus', 'and', 'sleep', 'under', 'the', 'Eiffel', 'Tower', '(', 'aka', 'Million', 'Stars', 'Hotel', ')', ':)', '.', '.', 'On', 'the', 'way', 'to', 'Paris', ',', 'I', "couldn't", 'sleep', 'because', 'of', 'excitement', '.', 'there', 'was', 'this', 'German', 'guy', 'who', 'was', 'sitting', 'right', 'next', 'to', 'me', '.', 'So', 'we', 'took', 'a', 'blanket', 'and', 'made', 'out', 'the', 'whole', 'night', ':)', '....']
[['token=Asslaam', 'char-1-gram=l s A a m', 'char-2-gram=sl la aa As am 

## Using feature generation function taking into account params in the string ngrams as defined in function
## Passing the generated feature string for training
## Defining a model using pycrfsuite module
## Defining models for different algorithms under sklearn_crfsuite module

In [39]:
%%time
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(train_features, train_labels):
  trainer.append(xseq, yseq)

trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 100,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

trainer.params()

trainer.train('model.crfsuite')

tagger = pycrfsuite.Tagger()
tagger.open('model.crfsuite')

pred_list = []

for index in range(len(valid_sents)):
  example_sent = valid_sents[index]
  pred_list.append(tagger.tag(sent2features_ngram(example_sent)))

print("Accuracy is {}".format(metrics.flat_accuracy_score(test_labels, pred_list)))
print("Precision in weighted mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='weighted')))
print("Recall in weighted mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='weighted')))
print("F1 score in weighted mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='weighted')))

print("\n\n")

print("Precision in micro mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='micro')))
print("Recall in micro mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='micro')))
print("F1 score in micro mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='micro')))

print("\n\n")

print("Precision in macro mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='macro')))
print("Recall in macro mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='macro')))
print("F1 score in macro mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='macro')))


Accuracy is 0.8665940124366946
Precision in weighted mode is 0.8889726206755119
Recall in weighted mode is 0.8665940124366946
F1 score in weighted mode is 0.8651708326072355



Precision in micro mode is 0.8665940124366946
Recall in micro mode is 0.8665940124366946
F1 score in micro mode is 0.8665940124366946



Precision in macro mode is 0.5822055742543365
Recall in macro mode is 0.446747006961452
F1 score in macro mode is 0.45567547467254876
CPU times: user 2.54 s, sys: 21.3 ms, total: 2.56 s
Wall time: 2.57 s


In [40]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',  ## Gradient descent using the L-BFGS method
    c1=0.1,
    c2=0.1,
    max_iterations=1000,
    all_possible_transitions=True
)

crf.fit(train_features, train_labels)

labels = list(crf.classes_)

y_pred = crf.predict(test_features)

print("Accuracy is {}".format(metrics.flat_accuracy_score(test_labels, y_pred)))
print("Precision in weighted mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='weighted')))
print("Recall in weighted mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='weighted')))
print("F1 score in weighted mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='weighted')))

print("\n\n")

print("Precision in micro mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='micro')))
print("Recall in micro mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='micro')))
print("F1 score in micro mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='micro')))

print("\n\n")

print("Precision in macro mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='macro')))
print("Recall in macro mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='macro')))
print("F1 score in macro mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='macro')))

Accuracy is 0.8687095326623502
Precision in weighted mode is 0.8889726206755119
Recall in weighted mode is 0.8665940124366946
F1 score in weighted mode is 0.8651708326072355



Precision in micro mode is 0.8665940124366946
Recall in micro mode is 0.8665940124366946
F1 score in micro mode is 0.8665940124366946



Precision in macro mode is 0.5822055742543365
Recall in macro mode is 0.446747006961452
F1 score in macro mode is 0.45567547467254876


In [41]:
crf = sklearn_crfsuite.CRF(
    algorithm='l2sgd', ## Stochastic Gradient Descent with L2 regularization term
    #c1=0.1,
    c2=0.1,
    max_iterations=1000,
    all_possible_transitions=True
)

crf.fit(train_features, train_labels)

labels = list(crf.classes_)

y_pred = crf.predict(test_features)

print("Accuracy is {}".format(metrics.flat_accuracy_score(test_labels, y_pred)))
print("Precision in weighted mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='weighted')))
print("Recall in weighted mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='weighted')))
print("F1 score in weighted mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='weighted')))

print("\n\n")

print("Precision in micro mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='micro')))
print("Recall in micro mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='micro')))
print("F1 score in micro mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='micro')))

print("\n\n")

print("Precision in macro mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='macro')))
print("Recall in macro mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='macro')))
print("F1 score in macro mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='macro')))



Accuracy is 0.8885184947753061
Precision in weighted mode is 0.8889726206755119
Recall in weighted mode is 0.8665940124366946
F1 score in weighted mode is 0.8651708326072355



Precision in micro mode is 0.8665940124366946
Recall in micro mode is 0.8665940124366946
F1 score in micro mode is 0.8665940124366946



Precision in macro mode is 0.5822055742543365
Recall in macro mode is 0.446747006961452
F1 score in macro mode is 0.45567547467254876


In [42]:
crf = sklearn_crfsuite.CRF(
    algorithm='ap', ## average perceptron
    #c1=0.1,
    #c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

crf.fit(train_features, train_labels)

labels = list(crf.classes_)

y_pred = crf.predict(test_features)

print("Accuracy is {}".format(metrics.flat_accuracy_score(test_labels, y_pred)))
print("Precision in weighted mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='weighted')))
print("Recall in weighted mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='weighted')))
print("F1 score in weighted mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='weighted')))

print("\n\n")

print("Precision in micro mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='micro')))
print("Recall in micro mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='micro')))
print("F1 score in micro mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='micro')))

print("\n\n")

print("Precision in macro mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='macro')))
print("Recall in macro mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='macro')))
print("F1 score in macro mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='macro')))

Accuracy is 0.8612731585358036
Precision in weighted mode is 0.8889726206755119
Recall in weighted mode is 0.8665940124366946
F1 score in weighted mode is 0.8651708326072355



Precision in micro mode is 0.8665940124366946
Recall in micro mode is 0.8665940124366946
F1 score in micro mode is 0.8665940124366946



Precision in macro mode is 0.5822055742543365
Recall in macro mode is 0.446747006961452
F1 score in macro mode is 0.45567547467254876


## Using feature generation function taking into account params in the string prefix, suffix as defined in function
## Passing the generated feature string for training
## Defining a model using pycrfsuite module
## Defining models for different algorithms under sklearn_crfsuite module

In [43]:
%%time
train_features = [sent2features_tags(sent) for sent in train_sents]
test_features = [sent2features_tags(sent) for sent in valid_sents]


print( train_features[0] )
print( test_features[0] )


[{'word': 'Asslaam', 'is_first': True, 'is_last': False, 'is_capitalized': False, 'is_all_caps': False, 'is_all_lower': False, 'prefix-1': 'A', 'prefix-2': 'As', 'prefix-3': 'Ass', 'suffix-1': 'm', 'suffix-2': 'am', 'suffix-3': 'aam', 'n-grams': ['A', 's', 's', 'l', 'a', 'a', 'm', 'As', 'ss', 'sl', 'la', 'aa', 'am', 'Ass', 'ssl', 'sla', 'laa', 'aam', 'Assl', 'ssla', 'slaa', 'laam'], 'prev_word': '', 'next_word': 'o', 'has_hyphen': False, 'is_numeric': False, 'capitals_inside': False}, {'word': 'o', 'is_first': False, 'is_last': False, 'is_capitalized': False, 'is_all_caps': False, 'is_all_lower': True, 'prefix-1': 'o', 'prefix-2': 'o', 'prefix-3': 'o', 'suffix-1': 'o', 'suffix-2': 'o', 'suffix-3': 'o', 'n-grams': ['o'], 'prev_word': 'Asslaam', 'next_word': "Alaikum!''-*Kehte", 'has_hyphen': False, 'is_numeric': False, 'capitals_inside': False}, {'word': "Alaikum!''-*Kehte", 'is_first': False, 'is_last': False, 'is_capitalized': False, 'is_all_caps': False, 'is_all_lower': False, 'prefi

In [44]:
%%time

trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(train_features, train_labels):
  trainer.append(xseq, yseq)
  
  
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 100,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

trainer.train('model1.crfsuite')

tagger = pycrfsuite.Tagger()
tagger.open('model1.crfsuite')

pred_list = []

for index in range(len(valid_sents)):
  example_sent = valid_sents[index]
  pred_list.append(tagger.tag(sent2features_tags(example_sent)))

print("Accuracy is {}".format(metrics.flat_accuracy_score(test_labels, pred_list)))
print("Precision in weighted mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='weighted')))
print("Recall in weighted mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='weighted')))
print("F1 score in weighted mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='weighted')))

print("\n\n")

print("Precision in micro mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='micro')))
print("Recall in micro mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='micro')))
print("F1 score in micro mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='micro')))

print("\n\n")

print("Precision in macro mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='macro')))
print("Recall in macro mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='macro')))
print("F1 score in macro mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='macro')))

Accuracy is 0.9156356176677992
Precision in weighted mode is 0.9303765355440051
Recall in weighted mode is 0.9156356176677992
F1 score in weighted mode is 0.9160566155723522



Precision in micro mode is 0.9156356176677992
Recall in micro mode is 0.9156356176677992
F1 score in micro mode is 0.9156356176677992



Precision in macro mode is 0.6085177655505432
Recall in macro mode is 0.5155658118912698
F1 score in macro mode is 0.5336748066264814
CPU times: user 5.9 s, sys: 20.6 ms, total: 5.93 s
Wall time: 5.93 s


In [45]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',  ## Gradient descent using the L-BFGS method
    c1=0.1,
    c2=0.1,
    max_iterations=1000,
    all_possible_transitions=True
)

crf.fit(train_features, train_labels)

labels = list(crf.classes_)

y_pred = crf.predict(test_features)

print("Accuracy is {}".format(metrics.flat_accuracy_score(test_labels, y_pred)))
print("Precision in weighted mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='weighted')))
print("Recall in weighted mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='weighted')))
print("F1 score in weighted mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='weighted')))

print("\n\n")

print("Precision in micro mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='micro')))
print("Recall in micro mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='micro')))
print("F1 score in micro mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='micro')))

print("\n\n")

print("Precision in macro mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='macro')))
print("Recall in macro mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='macro')))
print("F1 score in macro mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='macro')))

Accuracy is 0.9185204179755112
Precision in weighted mode is 0.9303765355440051
Recall in weighted mode is 0.9156356176677992
F1 score in weighted mode is 0.9160566155723522



Precision in micro mode is 0.9156356176677992
Recall in micro mode is 0.9156356176677992
F1 score in micro mode is 0.9156356176677992



Precision in macro mode is 0.6085177655505432
Recall in macro mode is 0.5155658118912698
F1 score in macro mode is 0.5336748066264814


In [46]:
crf = sklearn_crfsuite.CRF(
    algorithm='l2sgd', ## Stochastic Gradient Descent with L2 regularization term
    #c1=0.1,
    c2=0.1,
    max_iterations=1000,
    all_possible_transitions=True
)

crf.fit(train_features, train_labels)

labels = list(crf.classes_)

y_pred = crf.predict(test_features)

print("Accuracy is {}".format(metrics.flat_accuracy_score(test_labels, y_pred)))
print("Precision in weighted mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='weighted')))
print("Recall in weighted mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='weighted')))
print("F1 score in weighted mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='weighted')))

print("\n\n")

print("Precision in micro mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='micro')))
print("Recall in micro mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='micro')))
print("F1 score in micro mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='micro')))

print("\n\n")

print("Precision in macro mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='macro')))
print("Recall in macro mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='macro')))
print("F1 score in macro mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='macro')))

Accuracy is 0.9108917238284505
Precision in weighted mode is 0.9303765355440051
Recall in weighted mode is 0.9156356176677992
F1 score in weighted mode is 0.9160566155723522



Precision in micro mode is 0.9156356176677992
Recall in micro mode is 0.9156356176677992
F1 score in micro mode is 0.9156356176677992



Precision in macro mode is 0.6085177655505432
Recall in macro mode is 0.5155658118912698
F1 score in macro mode is 0.5336748066264814


In [47]:
crf = sklearn_crfsuite.CRF(
    algorithm='ap', ## average perceptron
    #c1=0.1,
    #c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

crf.fit(train_features, train_labels)

labels = list(crf.classes_)

y_pred = crf.predict(test_features)

print("Accuracy is {}".format(metrics.flat_accuracy_score(test_labels, y_pred)))
print("Precision in weighted mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='weighted')))
print("Recall in weighted mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='weighted')))
print("F1 score in weighted mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='weighted')))

print("\n\n")

print("Precision in micro mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='micro')))
print("Recall in micro mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='micro')))
print("F1 score in micro mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='micro')))

print("\n\n")

print("Precision in macro mode is {}".format(metrics.flat_precision_score(test_labels, pred_list, average='macro')))
print("Recall in macro mode is {}".format(metrics.flat_recall_score(test_labels, pred_list, average='macro')))
print("F1 score in macro mode is {}".format(metrics.flat_f1_score(test_labels, pred_list, average='macro')))


#print( test_labels[1])
#print( y_pred[1])

#print( test_features[1])
#print( valid_sents[1])

Accuracy is 0.9104429771139175
Precision in weighted mode is 0.9303765355440051
Recall in weighted mode is 0.9156356176677992
F1 score in weighted mode is 0.9160566155723522



Precision in micro mode is 0.9156356176677992
Recall in micro mode is 0.9156356176677992
F1 score in micro mode is 0.9156356176677992



Precision in macro mode is 0.6085177655505432
Recall in macro mode is 0.5155658118912698
F1 score in macro mode is 0.5336748066264814
